In [1]:
from urllib.request import urlopen
from urllib.parse import urlencode
from bs4 import BeautifulSoup as bs
from xml.etree import ElementTree as ET
import pandas as pd
import random
import re
import ssl

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [8]:
random.seed(50)

#links = set()

In [ ]:
def getRobots(initial_url):
    
    with urlopen(initial_url + '/robots.txt', context = ctx) as robot:
        text = robot.read().decode()
    links_to_avoid = set([s.split()[1] for s in text.split('\n') if s.startswith('Disallow:')])
    return links_to_avoid
    
def getLinks(pageUrl):
    
    html = urlopen(pageUrl, context = ctx)
    soup = bs(html)
      
    for link in soup.findAll('a', href = re.compile('^/book/show/')):
        if 'href' in link.attrs and re.findall(r'/book/show/[0-9]+', link.attrs['href'])[0] not in links_to_avoid:
            links.add(link.attrs['href'])
    
def nextLink(initial_url):
    url = initial_url + random.choice(tuple(links))
    return url

initial_url = 'https://www.goodreads.com'

links_to_avoid = getRobots(initial_url)

url = initial_url

for i in range(100):
    getLinks(url)
    url = nextLink(initial_url)
    if i % 20 == 0:
        print('Loop:', i)

Loop: 0
Loop: 50


In [6]:
len(links)

367

In [121]:
#html = urlopen(initial_url + '/book/show/2429135.The_Girl_with_the_Dragon_Tattoo', context = ctx)

def getInfo(book, item):
    try:
        return book.find('book').find(item).text
    except:
        return None
    
def getAuthors(book):
    try:
        return [t.find('./name').text for t in book.findall('./book/authors/author') if t.find('./role').text is None]
    except:
        return None
    
def getBookInfo(html):
    
    book = ET.fromstring(html.read())
    
    title = getInfo(book, 'title')
    book_id = getInfo(book, 'id')
    author = getAuthors(book)
    language = getInfo(book, 'language_code')
    rating = getInfo(book, 'average_rating')
    rating_count = getInfo(book, 'ratings_count')
    review_count = getInfo(book, 'text_reviews_count')
    pages = getInfo(book, 'num_pages')
    publication_year = getInfo(book, 'publication_year')
    
    return title, book_id, author, language, rating, rating_count, review_count, pages, publication_year

In [145]:
books = pd.DataFrame(books_list)
books.columns = ['title', 'book_id', 'author', 'language', 'rating', 'rating_count', 'review_count', 'pages', 'publication_year']

books.to_csv('books.csv', index = False)

In [144]:
display(books)

,title,book_id,author,language,rating,rating_count,review_count,pages,publication_year
0,Animals in Translation: Using the Mysteries of...,7366,"[Temple Grandin, Catherine Johnson]",eng,4.14,7761,980,358,2006
1,Water for Elephants,43641,[Sara Gruen],eng,4.09,1300046,53167,335,2007
2,The Book Thief,19063,[Markus Zusak],eng,4.37,1588577,89015,552,2006
3,Rebecca,1494702,[Daphne du Maurier],en-US,4.23,863,196,376,1948
4,My Cousin Rachel,34431455,[Daphne du Maurier],eng,3.97,716,155,400,2017
5,My Cousin Rachel,162543,[Daphne du Maurier],eng,3.97,1292,167,335,None
6,The Guernsey Literary and Potato Peel Pie Society,39832183,"[Mary Ann Shaffer, Annie Barrows]",eng,4.17,88753,5236,291,2018
7,Rebecca,17899948,[Daphne du Maurier],eng,4.23,206767,7850,449,2013
8,The Kite Runner,77203,[Khaled Hosseini],eng,4.30,2251096,61112,371,2004
9,Rebecca,636993,[Daphne du Maurier],en-US,4.23,6430,1033,380,2002
